<a href="https://colab.research.google.com/github/KimGriezmann/Bigcontest_2021/blob/master/working_space_JaeHong/create_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [115]:
import pandas as pd
import numpy as np

ops_2018 = pd.read_csv('/content/drive/MyDrive/bigcontest/final_ops_data_2018.csv', encoding='euc-kr')
ops_2019 = pd.read_csv('/content/drive/MyDrive/bigcontest/final_ops_data_2019.csv', encoding='euc-kr')
ops_2020 = pd.read_csv('/content/drive/MyDrive/bigcontest/final_ops_data_2020.csv', encoding='euc-kr')
ops_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/final_ops_data_2021.csv', encoding='euc-kr')
barrel_df = pd.read_csv('/content/drive/MyDrive/bigcontest/barrel_df.csv', encoding='euc-kr')

In [116]:
barrel_df

,PCODE,DATE,G_ID,power_barrel,contact_barrel,clutch_barrel
0,50054,20200603,0,0.0,2.0,0.0
1,50054,20200604,0,0.0,0.0,0.0
2,50054,20200607,0,0.0,1.0,0.0
3,50054,20200611,0,0.0,0.0,0.0
4,50054,20200613,1,0.0,1.0,0.0
...,...,...,...,...,...,...
49772,99810,20181006,0,0.0,1.0,0.0
49773,99810,20181009,0,0.0,1.0,0.0
49774,99810,20181010,1,0.0,0.0,0.0
49775,99810,20181012,0,2.0,0.0,0.0


In [117]:
# 날짜 데이터 값, 형식 변경
def day_to_int(data):
  data['day'] = pd.to_numeric(data['day'].str.replace("-",""))
  return data

In [118]:
ops_2021 = day_to_int(ops_2021)
ops_2020 = day_to_int(ops_2020)
ops_2019 = day_to_int(ops_2019)
ops_2018 = day_to_int(ops_2018)

In [119]:
#ops 데이터에 더블헤더(order) 처리
# 더블헤더 처리
def order_double(data):
  temp = np.zeros(len(data))

  # day, name이 같은 값이 있다는 것은 더블헤더 경기가 있다는것
  # keep=False 옵션은 중복 값에 해당하는 튜플들을 모두 True로 리턴, 그 외 옵션 : 'first', 'last' 의 경우 둘 중 하나만 True
  dup = data.loc[:,['day','name']].duplicated(keep=False) 

  for i in range(len(data)):
    if dup[i]==False:
      temp[i]=0
    elif dup[i]==True:
      temp[i]=1

  # 중복의 첫번째 행은 더블헤더 1차전
  # 중복인 두번째 행이 더블헤더 2차전이다.
  for i in range(1,len(temp)):
    if temp[i]==1 and temp[i-1]==1:
      temp[i]=2

  data['G_ID']=temp

  return data

In [120]:
ops_2021 = order_double(ops_2021)
ops_2020 = order_double(ops_2020)
ops_2019 = order_double(ops_2019)
ops_2018 = order_double(ops_2018)

In [123]:
ops_2021.rename(columns={'day':'DATE'}, inplace=True)
ops_2020.rename(columns={'day':'DATE'}, inplace=True)
ops_2019.rename(columns={'day':'DATE'}, inplace=True)
ops_2018.rename(columns={'day':'DATE'}, inplace=True)

In [125]:
add_hts_2021 = ops_2021.merge(barrel_df, on=['PCODE', 'DATE', 'G_ID'])
add_hts_2020 = ops_2020.merge(barrel_df, on=['PCODE', 'DATE', 'G_ID'])
add_hts_2019 = ops_2019.merge(barrel_df, on=['PCODE', 'DATE', 'G_ID'])
add_hts_2018 = ops_2018.merge(barrel_df, on=['PCODE', 'DATE', 'G_ID'])

In [139]:
add_hts_2021.to_csv("add_hts_2021.csv", encoding='euc-kr', index=False)
add_hts_2020.to_csv("add_hts_2020.csv", encoding='euc-kr', index=False)
add_hts_2019.to_csv("add_hts_2019.csv", encoding='euc-kr', index=False)
add_hts_2018.to_csv("add_hts_2018.csv", encoding='euc-kr', index=False)